In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings

warnings.filterwarnings('ignore')

# Enhanced Academic publication style settings
plt.style.use('seaborn-v0_8-paper')
sns.set_context("paper", font_scale=1.6)
sns.set_palette("deep")

# Configure matplotlib for publication quality with BOLD and BIGGER fonts
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman', 'DejaVu Serif']
plt.rcParams['font.size'] = 13
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 17
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.edgecolor'] = 'black'
plt.rcParams['legend.title_fontsize'] = 13
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['xtick.major.width'] = 1.2
plt.rcParams['ytick.major.width'] = 1.2

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.precision', 4)

print("Libraries loaded successfully!")
print("Enhanced academic publication style configured")
print("Experiment 2: WITH Artist Features (414 features)")

In [ ]:
# Load Experiment 2 results (with artist features)
df = pd.read_csv('../results/metrics/experiment2_with_artist/enhanced_results_detailed_20251231_143812.csv')
print(f"Loaded {len(df)} model results from Experiment 2")
print(f"Targets: {df['target'].unique().tolist()}")
print(f"Models: {df['model'].nunique()} unique models")
print(f"Features: 414 (23 audio + 5 text + 2 sentiment + 384 embeddings)")
print(f"Artist Features: log_total_artist_followers + avg_artist_popularity")

---
## 🏆 Best Models Summary (Quick Overview)

In [ ]:
# Find best model for each target
best_models = df.loc[df.groupby('target')['r2'].idxmax()][['target', 'model', 'r2', 'rmse', 'mae']]
best_models = best_models.sort_values('r2', ascending=False)

# Style the dataframe
def highlight_best(s):
    return ['background-color: #90EE90; font-weight: bold' if i == s.name else '' for i in range(len(s))]

styled_best = best_models.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'mae': '{:.4f}'
}).set_caption('🏆 Best Model Per Target').set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '18px'), ('font-weight', 'bold'), ('color', '#2E86AB')]}
])

display(styled_best)

---
## 🎭 Valence Models (Emotional Positivity)

In [ ]:
def create_styled_table(df, target, sort_by='r2', ascending=False):
    """Create a beautifully styled table for a specific target"""
    target_df = df[df['target'] == target].copy()
    target_df = target_df.sort_values(sort_by, ascending=ascending)
    
    # Select key columns
    cols = ['model', 'r2', 'rmse', 'mae', 'explained_variance', 'train_time_seconds']
    target_df = target_df[cols].reset_index(drop=True)
    target_df['rank'] = range(1, len(target_df) + 1)
    target_df = target_df[['rank'] + cols]
    
    # Color coding function
    def color_r2(val):
        if val >= 0.6:
            return 'background-color: #90EE90'  # Green
        elif val >= 0.4:
            return 'background-color: #FFFFE0'  # Light yellow
        elif val >= 0.2:
            return 'background-color: #FFD700'  # Gold
        elif val >= 0:
            return 'background-color: #FFA500'  # Orange
        else:
            return 'background-color: #FF6B6B'  # Red
    
    styled = target_df.style.format({
        'r2': '{:.4f}',
        'rmse': '{:.4f}',
        'mae': '{:.4f}',
        'explained_variance': '{:.4f}',
        'train_time_seconds': '{:.1f}s'
    }).applymap(color_r2, subset=['r2']).set_caption(
        f'{target.upper()} Models - Sorted by R²'
    ).set_table_styles([
        {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
        {'selector': 'th', 'props': [('background-color', '#4A90D9'), ('color', 'white')]},
    ])
    
    return styled

display(create_styled_table(df, 'valence'))

---
## ⚡ Energy Models (Intensity)

In [ ]:
display(create_styled_table(df, 'energy'))

---
## 💃 Danceability Models

In [ ]:
display(create_styled_table(df, 'danceability'))

---
## 🌟 Popularity Models (Hardest Target)

In [ ]:
display(create_styled_table(df, 'popularity'))

---
## 📈 Visual Comparison: R² Scores Across All Targets

In [ ]:
# Create R² comparison heatmap with enhanced academic styling
pivot_r2 = df.pivot(index='model', columns='target', values='r2')
pivot_r2 = pivot_r2[['valence', 'energy', 'danceability', 'popularity']]  # Order columns

# Sort by average R²
pivot_r2['avg_r2'] = pivot_r2.mean(axis=1)
pivot_r2 = pivot_r2.sort_values('avg_r2', ascending=False)
pivot_r2 = pivot_r2.drop('avg_r2', axis=1)

# Create heatmap with enhanced styling
fig, ax = plt.subplots(figsize=(14, 16))
sns.heatmap(pivot_r2, annot=True, fmt='.3f', cmap='RdYlGn', center=0.3,
            linewidths=2, linecolor='white', ax=ax, vmin=-0.2, vmax=0.9,
            cbar_kws={'label': 'R² Score', 'shrink': 0.85},
            annot_kws={'fontsize': 11, 'weight': 'bold'})
ax.set_title('R² Scores Heatmap', 
             fontsize=18, fontweight='bold', pad=25)
ax.tick_params(axis='x', labelsize=14, width=1.5)
ax.tick_params(axis='y', labelsize=13, width=1.5)
plt.setp(ax.get_xticklabels(), fontweight='bold')
plt.setp(ax.get_yticklabels(), fontweight='bold')
plt.tight_layout()
plt.savefig('../results/figures/experiment2_r2_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_r2_heatmap.png")

---
## 📊 Top 5 Models Bar Chart Per Target

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
targets = ['valence', 'energy', 'danceability', 'popularity']
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D']

for ax, target, color in zip(axes.flatten(), targets, colors):
    top5 = df[df['target'] == target].nlargest(5, 'r2')
    
    bars = ax.barh(top5['model'], top5['r2'], color=color, edgecolor='black', alpha=0.85, linewidth=1.5)
    ax.set_xlabel('R² Score', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()}', fontsize=15, fontweight='bold', pad=15)
    ax.set_xlim(0, max(top5['r2']) * 1.15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2, labelsize=12)
    
    # Add value labels with bold font
    for bar, val in zip(bars, top5['r2']):
        ax.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.3f}', 
                va='center', fontsize=11, fontweight='bold')
    
    ax.invert_yaxis()
    plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)

plt.suptitle('Top 5 Models by R² Score', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_top5_models.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_top5_models.png")

---
## ⏱️ Training Time Analysis

In [ ]:
# Aggregate training time by model with enhanced styling
time_by_model = df.groupby('model')['train_time_seconds'].mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(14, 10))
colors = plt.cm.viridis(np.linspace(0, 1, len(time_by_model)))
bars = ax.barh(time_by_model.index, time_by_model.values, color=colors, 
               edgecolor='black', alpha=0.85, linewidth=1.5)
ax.set_xlabel('Average Training Time (seconds)', fontsize=15, fontweight='bold', labelpad=10)
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)
ax.tick_params(axis='both', width=1.2, labelsize=12)

# Add value labels
for bar, val in zip(bars, time_by_model.values):
    if val > 100:
        label = f'{val/60:.1f}m'
    else:
        label = f'{val:.1f}s'
    ax.text(val + 10, bar.get_y() + bar.get_height()/2, label, 
            va='center', fontsize=11, fontweight='bold')

plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_training_time.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_training_time.png")

---
## 📉 RMSE Comparison

In [ ]:
# Create RMSE comparison (lower is better) with enhanced styling
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for ax, target, color in zip(axes.flatten(), targets, colors):
    top5 = df[df['target'] == target].nsmallest(5, 'rmse')  # Smallest RMSE = best
    
    bars = ax.barh(top5['model'], top5['rmse'], color=color, edgecolor='black', 
                   alpha=0.85, linewidth=1.5)
    ax.set_xlabel('RMSE', fontsize=14, fontweight='bold', labelpad=8)
    ax.set_title(f'{target.upper()}', fontsize=15, fontweight='bold', pad=15)
    ax.grid(True, alpha=0.3, axis='x', linestyle='--', linewidth=1.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.tick_params(axis='both', width=1.2, labelsize=12)
    
    # Add value labels
    for bar, val in zip(bars, top5['rmse']):
        ax.text(val + 0.001, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
                va='center', fontsize=11, fontweight='bold')
    
    ax.invert_yaxis()
    plt.setp(ax.get_yticklabels(), fontweight='bold', fontsize=12)

plt.suptitle('Top 5 Models by RMSE', 
             fontsize=17, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/experiment2_top5_rmse.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_top5_rmse.png")

---
## 🔬 Default vs Tuned Models Comparison

In [ ]:
# Compare default vs tuned versions
tuned_models = df[df['model'].str.contains('_tuned')].copy()
tuned_models['base_model'] = tuned_models['model'].str.replace('_tuned', '')

default_models = df[~df['model'].str.contains('_tuned') & ~df['model'].isin(['Mean'])].copy()
default_models['base_model'] = default_models['model']

# Merge to compare
comparison = tuned_models.merge(
    default_models[['target', 'base_model', 'r2', 'rmse']], 
    on=['target', 'base_model'], 
    suffixes=('_tuned', '_default')
)
comparison['r2_improvement'] = comparison['r2_tuned'] - comparison['r2_default']
comparison['rmse_improvement'] = comparison['rmse_default'] - comparison['rmse_tuned']  # Positive = better

# Display improvement summary
improvement_summary = comparison.groupby('base_model').agg({
    'r2_improvement': 'mean',
    'rmse_improvement': 'mean'
}).sort_values('r2_improvement', ascending=False)

print("Tuning Impact (Average across all targets):")
print("="*50)
styled_improvement = improvement_summary.style.format({
    'r2_improvement': '{:+.4f}',
    'rmse_improvement': '{:+.4f}'
}).background_gradient(cmap='RdYlGn', subset=['r2_improvement']).set_caption(
    'Tuning Impact: Positive = Improvement'
)
display(styled_improvement)

In [ ]:
# Visualize tuning impact with enhanced styling
fig, ax = plt.subplots(figsize=(14, 7))

x = np.arange(len(improvement_summary))
width = 0.6

bars = ax.bar(x, improvement_summary['r2_improvement'], width, 
              color=['#90EE90' if v > 0 else '#FF6B6B' for v in improvement_summary['r2_improvement']],
              edgecolor='black', alpha=0.85, linewidth=1.5)

ax.set_ylabel('R² Improvement', fontsize=15, fontweight='bold', labelpad=10)
ax.set_title(' Hyperparameter Tuning Impact on R²', 
             fontsize=17, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(improvement_summary.index, rotation=45, ha='right', fontweight='bold', fontsize=12)
ax.axhline(y=0, color='black', linestyle='-', linewidth=2)
ax.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_linewidth(1.5)
ax.spines['bottom'].set_linewidth(1.5)
ax.tick_params(axis='both', width=1.2, labelsize=12)

# Add value labels with enhanced visibility
for bar, val in zip(bars, improvement_summary['r2_improvement']):
    y_pos = val + 0.003 if val >= 0 else val - 0.012
    ax.text(bar.get_x() + bar.get_width()/2, y_pos, f'{val:+.3f}', 
            ha='center', va='bottom' if val >= 0 else 'top', 
            fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('../results/figures/experiment2_tuning_impact.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved: results/figures/experiment2_tuning_impact.png")

---
## 📋 Complete Results Table (All Models, All Targets)

In [ ]:
# Create comprehensive table
full_table = df[['target', 'model', 'r2', 'rmse', 'mae', 'explained_variance', 
                 'pred_mean', 'pred_std', 'residual_mean', 'residual_std', 'train_time_seconds']].copy()
full_table = full_table.sort_values(['target', 'r2'], ascending=[True, False])

def color_by_target(val):
    colors = {
        'valence': 'background-color: #E8F4FD',
        'energy': 'background-color: #FDF4E8',
        'danceability': 'background-color: #F4E8FD',
        'popularity': 'background-color: #E8FDF4'
    }
    return colors.get(val, '')

styled_full = full_table.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'mae': '{:.4f}',
    'explained_variance': '{:.4f}',
    'pred_mean': '{:.4f}',
    'pred_std': '{:.4f}',
    'residual_mean': '{:.4f}',
    'residual_std': '{:.4f}',
    'train_time_seconds': '{:.1f}'
}).applymap(color_by_target, subset=['target']).set_caption(
    'Complete Results - All Models, All Targets'
).set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '16px'), ('font-weight', 'bold')]},
    {'selector': 'th', 'props': [('background-color', '#2E86AB'), ('color', 'white')]},
])

display(styled_full)

---
## 🎯 Final Recommendations

In [ ]:
# Get top 2 models per target
recommendations = []
for target in targets:
    top2 = df[df['target'] == target].nlargest(2, 'r2')[['target', 'model', 'r2', 'rmse', 'train_time_seconds']]
    top2['recommendation'] = ['🥇 Primary', '🥈 Backup']
    recommendations.append(top2)

rec_df = pd.concat(recommendations)

styled_rec = rec_df.style.format({
    'r2': '{:.4f}',
    'rmse': '{:.4f}',
    'train_time_seconds': '{:.1f}s'
}).set_caption('Final Model Recommendations for Test Evaluation').set_table_styles([
    {'selector': 'caption', 'props': [('font-size', '18px'), ('font-weight', 'bold'), ('color', '#2E86AB')]},
    {'selector': 'th', 'props': [('background-color', '#2E86AB'), ('color', 'white')]},
])

display(styled_rec)

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for target in targets:
    best = df[df['target'] == target].nlargest(1, 'r2').iloc[0]
    print(f"\n{target.upper()}:")
    print(f"  Best Model: {best['model']}")
    print(f"  R² = {best['r2']:.4f} | RMSE = {best['rmse']:.4f}")